In [1]:
import ROOT as r
r.EnableImplicitMT()

from xgboost import XGBClassifier, plot_importance
import numpy as np
import pandas as pd

from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

import hist
from matplotlib import pyplot as plt
import mplhep as hep
hep.style.use(hep.style.CMS)

In [2]:
def load_data(signal_filename, background_filename):
    variables = ["VBSjet1pt", "VBSjet1eta", "VBSjet1phi", "VBSjet2pt", "VBSjet2eta", "VBSjet2phi", "VBSMjj", "VBSdetajj", "weight"]

    def pddf(filename, variables):
        data = r.RDataFrame("Events", filename).Filter("weight > 0").Filter("passCut9").AsNumpy(variables)
        return pd.DataFrame(data)
    
    x_sig = pddf(signal_filename, variables)
    x_bkg = pddf(background_filename, variables)

    x_sig["weight"] = x_sig["weight"] / max(x_sig["weight"])
    x_bkg["weight"] = x_bkg["weight"] / max(x_bkg["weight"])

    x_sig["label"] = 1
    x_bkg["label"] = 0

    df = pd.concat([x_sig, x_bkg], ignore_index=True)
    df = df.sample(frac=1).reset_index(drop=True)

    return df

In [ ]:
df = load_data("/data/userdata/aaarora/output/run2/sig.root", "/data/userdata/aaarora/output/run2/bkg.root")

X = df.drop(columns=["label", "weight"]).to_numpy()
y = df["label"].to_numpy()
w = df["weight"].to_numpy()

In [ ]:
X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(X, y, w, test_size=0.2, random_state=42)

In [4]:
params = {
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "tree_method": "exact",
    "n_estimators": 200,
    "max_depth": 10,
    "min_child_weight": 1,
    "colsample_bytree": 1,
    "colsample_bylevel": 1,
    "subsample": 0.8,
    "learning_rate": 0.1,
    "lambda": 0.1,
    "alpha": 0.1,
    "gamma": 0,
    "nthread": 128,
}
bdt = XGBClassifier(**params)

In [ ]:
bdt.fit(X_train, y_train, sample_weight=w_train, eval_set=[(X_test, y_test)], verbose=True)

In [6]:
y_pred = bdt.predict_proba(X_test)[:, 1]

In [7]:
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, y_pred)
score = auc(false_positive_rate, true_positive_rate)

In [ ]:
fig, ax = plt.subplots()
ax.plot(false_positive_rate, true_positive_rate, label='ROC curve (area = %0.2f)' % score)
ax.plot([0, 1], [0, 1], 'k--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title("ROC After Hbb and Vqq Score Cuts")
ax.legend()

plt.savefig("/home/users/aaarora/public_html/vbs/plots/bdt-9-19-24/roc-curve-after-cut.png")

In [ ]:
plot_importance(bdt, max_num_features=None)

In [10]:
r.TMVA.Experimental.SaveXGBoost(bdt, "VBSBDT", "BDT_Weights.root", num_inputs=X.shape[1])

# Validation

In [11]:
bdt = r.TMVA.Experimental.RBDT("VBSBDT", "/home/users/aaarora/phys/analysis/vbs-1lep/training/BDT/BDT_Weights.root")

In [ ]:
h1 = hist.Hist.new.Reg(50, 0, 1, name="SIG", label="SIG").Double()
h1.fill(bdt.Compute(X[y==1]).flatten(), weight=w[y==1])

h2 = hist.Hist.new.Reg(50, 0, 1, name="BKG", label="BKG").Double()
h2.fill(bdt.Compute(X[y==0]).flatten(), weight=w[y==0])

fig, ax = plt.subplots()
hep.histplot(h1, ax=ax, label="Signal", color="red", density=True)
hep.histplot(h2, ax=ax, label="Background", color="blue", density=True)

ax.legend()
ax.set_xlabel("VBSBDT Score")
ax.set_ylabel("Density")
ax.set_title("Sig vs Bkg BDT Score Distribution After Cuts")

plt.savefig("/home/users/aaarora/public_html/vbs/plots/bdt-9-19-24/score-dists-after-cut.png")